In [14]:
import numpy as np
import pandas as pd

import tensorflow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from tensorflow.keras.layers import InputLayer,Dense
from tensorflow.keras.models import Sequential

In [15]:
ecg=pd.read_csv("Dataset/ECGdataset/ecg_autoencoder_dataset.csv")
X=ecg.values
y=np.zeros(X.shape[0])

In [16]:
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)

In [18]:
input_dim,encoding_dim=xtrain.shape[1],16

In [19]:
encoder=Sequential([
    InputLayer(input_shape=(input_dim,)),
    Dense(32,activation="relu"),
    Dense(encoding_dim,activation="relu")
])

In [20]:
decoder=Sequential([
    InputLayer(input_shape=(encoding_dim,)),
    Dense(32,activation="relu"),
    Dense(input_dim,activation="sigmoid")
])

In [21]:
autoencode=Sequential([encoder,decoder])
autoencode.compile(optimizer="adam",loss="mse")
autoencode.fit(xtrain,xtrain,epochs=10,batch_size=32,shuffle=True,validation_data=(xtest,xtest))

autoencode.summary()

Epoch 1/10
125/125 [==============================] - 1s 4ms/step - loss: 0.9394 - val_loss: 0.8397
Epoch 2/10
125/125 [==============================] - 0s 3ms/step - loss: 0.7325 - val_loss: 0.7819
Epoch 3/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6968 - val_loss: 0.7570
Epoch 4/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6810 - val_loss: 0.7464
Epoch 5/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6719 - val_loss: 0.7407
Epoch 6/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6666 - val_loss: 0.7360
Epoch 7/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6622 - val_loss: 0.7320
Epoch 8/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6587 - val_loss: 0.7284
Epoch 9/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6556 - val_loss: 0.7264
Epoch 10/10
125/125 [==============================] - 0s 3ms/step - loss: 0.6537 - val_loss: 0.7245

In [22]:
prediction=autoencode.predict(xtest)
mse=np.mean((xtest-prediction)**2,axis=1)
threshold=np.percentile(mse,95)

y_pred=np.where(mse > threshold,1,0)

32/32 [==============================] - 0s 2ms/step


In [23]:
accuracy=accuracy_score(ytest,y_pred)
accuracy

0.95